In [1]:
import pandas as pd

In [2]:
overdrive_fh  = "./reports/BPL/od-bpl-all.csv"

In [3]:
odf_raw = pd.read_csv(overdrive_fh, header=None, names=["overdriveNo", "overdriveControlNo", "sierraFormat_ext"])

In [4]:
odf_raw.head()

,overdriveNo,overdriveControlNo,sierraFormat_ext
0,0E90D7A5-30B8-4D07-9D13-DF0E02EA631E,ODN0000082587,z
1,C449DE3C-A683-412A-8997-F0551A5DC576,ODN0000082663,z
2,765A1B6A-E978-4777-8DD1-9647D611A58B,ODN0000082589,z
3,0957DA4E-DD2F-469E-BDFF-B9B1A9422C21,ODN0000082590,z
4,E572EC2C-09F7-462D-8153-7C8A7493586F,ODN0000082591,z


In [5]:
# find if any missing IDs
odf_missing_ids = odf_raw[odf_raw["overdriveNo"].isnull()]

In [6]:
odf_missing_ids.shape

(0, 3)

In [7]:
odf_raw["sierraFormat_ext"].value_counts()

x    171343
z     43806
v      1985
Name: sierraFormat_ext, dtype: int64

In [8]:
sierra_fh = "./reports/BPL/sierra-bpl-all.csv"

In [9]:
sdf_raw = pd.read_csv(
    sierra_fh,
    header=None,
    names=["overdriveNo", "overdriveNoSrc", "bibNo", "sierraControlNo", "sierraControlNoSrc", "sierraFormat_int", "sierra_status"],
    low_memory=False
)

In [10]:
sdf_raw.head()

,overdriveNo,overdriveNoSrc,bibNo,sierraControlNo,sierraControlNoSrc,sierraFormat_int,sierra_status
0,27C3C666-343B-442B-A81C-2068E6D1A664,037,b112402306,ocm54908102,OCoLC,x,-
1,C1AC2A72-58D4-4611-9F80-00CFE226CF2C,037,b112402318,ocm55680353,OCoLC,x,-
2,8CD53ED9-CEBD-4F78-8BEF-20A58F6F3857,037,b11240232x,ocm56833247,OCoLC,x,-
3,CAC65044-3B9C-4B12-B80B-00D9A09ABAB8,037,b112402331,ocm57443259,OCoLC,x,-
4,3D783A28-2DA9-4798-A3C2-1F65D2208F38,037,b112402343,ocm60397130,OCoLC,x,-


In [11]:
sdf_raw["sierraFormat_int"].value_counts()

x    161347
z     41264
v       935
s        45
Name: sierraFormat_int, dtype: int64

In [12]:
# sierra fomat "s"?
sdf_s_format = sdf_raw[sdf_raw["sierraFormat_int"] == "s"]

In [13]:
sdf_s_format.head()

,overdriveNo,overdriveNoSrc,bibNo,sierraControlNo,sierraControlNoSrc,sierraFormat_int,sierra_status
713,15B32E52-357A-4D74-A43E-18C673B0B0FC,url,b112467854,ocm57507474,OCoLC,s,-
714,EBC278AA-D4EA-45FB-8B3B-DFDF2122ADA0,url,b112467866,ocm57507588,OCoLC,s,-
715,35AC0D13-E37C-47E0-8DC9-DEEFC69D10DF,url,b112467878,ocm57507915,OCoLC,s,-
717,7B5DF2A5-6C03-495A-B98B-4A6B49D7BF6A,url,b112467891,ocm57508902,OCoLC,s,-
718,679C31DB-8F49-4A59-B697-C0CE35CDD490,url,b112467908,ocm57509118,OCoLC,s,-


## Inner join between Sierra and bulk MarcExpress - records already in the catalog

In [14]:
# this looks like legitimate bibs that are marked differently in Sierra than on Overload platform

In [15]:
df_available = pd.merge(odf_raw, sdf_raw, on="overdriveNo", how="inner")

In [16]:
df_available.head()

,overdriveNo,overdriveControlNo,sierraFormat_ext,overdriveNoSrc,bibNo,sierraControlNo,sierraControlNoSrc,sierraFormat_int,sierra_status
0,0E90D7A5-30B8-4D07-9D13-DF0E02EA631E,ODN0000082587,z,url,b113910861,ocm63791051,OCoLC,z,-
1,C449DE3C-A683-412A-8997-F0551A5DC576,ODN0000082663,z,037,b122634597,ocm63680586,NaN,z,-
2,765A1B6A-E978-4777-8DD1-9647D611A58B,ODN0000082589,z,url,b113851571,ocm63791673,OCoLC,z,-
3,0957DA4E-DD2F-469E-BDFF-B9B1A9422C21,ODN0000082590,z,037,b113026158,ocm66900955,OCoLC,z,-
4,E572EC2C-09F7-462D-8153-7C8A7493586F,ODN0000082591,z,037,b11302616x,ocm66900982,OCoLC,z,-


In [17]:
df_available.shape

(189051, 9)

#### Suppressed/marked for deletion bibs in Sierra to be reinstated

In [21]:
# find records that are in the catalog, users have access to, but are suppressed
df_available["sierra_status"].value_counts()

-    188902
n       127
d        22
Name: sierra_status, dtype: int64

In [26]:
df_reinstate_access = df_available[(df_available["sierra_status"]=="n")|(df_available["sierra_status"]=="d")]

In [27]:
df_reinstate_access.shape

(149, 9)

In [28]:
df_reinstate_access.head()

,overdriveNo,overdriveControlNo,sierraFormat_ext,overdriveNoSrc,bibNo,sierraControlNo,sierraControlNoSrc,sierraFormat_int,sierra_status
1660,31C31E3A-EE4F-499F-98F8-5987FBD715D1,ODN0003566156,z,037,b121446918,on1042160321,NaN,z,n
3519,4D21F8F0-100A-4C25-941D-B5D480091625,ODN0003566139,z,037,b121390846,on1033674500,NaN,z,n
6696,1C0B9DC4-653A-49F7-9187-2176F6BA859D,ODN0000060795,z,url,b112467064,ocm57398773,OCoLC,z,n
7209,A2D8D0EA-38D5-434E-994C-036393218ACD,ODN0000155769,z,037,b118440718,ocn455436062,OCoLC,z,d
8826,6F245F1F-05C7-4C5C-84C1-A57DE8E92A38,ODN0000063301,z,037,b117856587,ocm59755950,OCoLC,z,n


In [29]:
df_reinstate_access.to_csv("./reports/BPL/reinstate_access-alldata.csv", index=False)

In [30]:
df_reinstate_access["bibNo"].to_csv("./reports/BPL/reinstate_access-bibNos", index=False)